In [49]:
import csv
import numpy as np
import ast
import json
import os
import sys
import re
from textblob import TextBlob
import collections

In [3]:
def featuresExtraction(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    features = []
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    
    return np.array(features)

In [4]:
def labelExtraction(label_file):
#     csv_file = open(file)
#     csv_reader = csv.reader(csv_file, delimiter = ' ')
#     labels = []
#     for row in csv_reader:
#         labels.append(row[0])
    labels = np.genfromtxt(label_file,dtype=None)
    return labels

In [5]:
def reduction(array, key):
    reducedArray = []
    for i, j in enumerate(array):
        if len(reducedArray) == 0:
            reducedArray.append(j)
        else:
            count = 0
            for k in range(len(reducedArray)):
                if a[k] != b[i]:
                    count = count + 1
            
                if count == len(a):
                    a.append(b[i])
    
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    return reducedArray

In [6]:
def getCredentials():
    app_key = "jGVy1ilV55A7P24eJHuousM1b"
    app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
    oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
    oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
    
    return app_key, app_secret, oauth_token, oauth_token_secret

In [7]:
def featuresExtraction2(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    final = []
    textFeatures = []
    
    for i, j in enumerate(convertedArray):
        tempDict = {}
        tempDict['id'] = j['id']
        tempDict['date'] = j['created_at']
        tempDict['username'] = j['user']['screen_name']
        tempDict['hashtags'] = j['entities']['hashtags']
        tempDict['user_mentions'] = j['entities']['user_mentions']
        tempDict['favorites'] = j['favorite_count']
        
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            textFeatures.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            textFeatures.append(j['extended_tweet']['full_text'])
        else:
            textFeatures.append(j['text'])
        
        final.append(tempDict)
    
    return final, textFeatures

In [8]:
def authorization(key):
    if key == 'e' or 'E':
        app_key = "jGVy1ilV55A7P24eJHuousM1b"
        app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
        oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
        oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
        return app_key, app_secret, oauth_token, oauth_token_secret
    elif key == 'v' or 'V':
        app_key = "d"
    else:
        print("Wrong identification key")

In [9]:
def cred_Processing(tweetsFile):
    idRegEx = re.compile(r".*ID=")
    endElRegEX = re.compile(r"'.*")
    output = open("outputTweets.json", "w")
    
    with open(tweetsFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            
            topicKey = topicData[0]
            topicTerms = topicData[1]
            topicTweetCount = topicData[2]
            tweetIdList = topicData[3]
            
            print(topicKey)
            
            realTweetsIds = []
            
            idElements = tweetIdList.split("),")
            for element in idElements:
                elArr = element.split(",")
                idEl = list(filter(lambda x: "ID" in x, elArr))[0]
                idEl = idRegEx.sub("", idEl)
                idEl = endElRegEX.sub("", idEl)
                
                realTweetsIds.append(int(idEl))
                
            realTweetsIds = list(set(realTweetsIds))
            
            topicMap = {
                "key" : topicKey,
                "terms" : topicTerms.split(","),
                "count" : topicTweetCount, 
                "tweets" : realTweetsIds
            }
            
            json.dump(topicMap, output, sort_keys=True)
            output.write("\n")
    output.close()

In [23]:
def rating_processing(ratingFile, tweetsFile):
    tweetsMap = {}
    
    with open(tweetsFile, "r") as f:
        for line in f:
            tweetData = json.loads(line)
            tweetsMap[tweetData["key"]] = tweetData
            
    output = open(r"C:\Users\eric4\Desktop\University\Research\Data\CREDBANK\finalOutput.json", "w")
    
    with open(ratingFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            topicKey = topicData[0]
            topicTerms = topicData[1]
            ratings = topicData[2]
            reasons = topicData[3]
            
            ratings = list(map(lambda x: int(x.strip().replace("'", "")), ratings.replace("[", "").replace("]", "").split(",")))
            ratings = np.array(ratings)
            tweetsMap[topicKey]["mean"] = np.mean(ratings)
            try:
                print("Inserting ratings for {}".format(topicKey))
                tweetsMap[topicKey]["ratings"] = ratings.tolist()
                print("Success")
                print("Inserting mean for {}".format(topicKey))
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Success")
                topicMap = tweetsMap[topicKey]
                print(topicMap["key"], topicMap["mean"])
                json.dump(topicMap, output, sort_keys=True)
                output.write("\n")
            except:
                print("{} is missing.".format(topicKey))
            
            
            
    output.close()

In [35]:
def binaryConversion(outputFile):
    labels = {}
    with open(outputFile, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            if(tweet['mean'] >= 1.9):
                labels[tweet['key']] = 1
            elif(tweet['mean'] <= 1.467):
                labels[tweet['key']] = 0
    return labels

In [3]:
def messageFeatureExtraction(tweets, threshold):
    tweetsFeatures = []
    np.array(tweetsFeatures)
    count = 1
    for tweet in tweets:
        print("Starting feature extraction for tweet {} of {}".format(count, len(tweets)))
        indiTweetFeature = []
        #Length of characters 1
        print("Adding Length of charactrs")
        indiTweetFeature.append(len(tweet))
        #print(indiTweetFeature)
        
        print("Adding words")
        #Length of words 2
        indiTweetFeature.append(len(tweet.split(" ")))
        #print(indiTweetFeature)
        
        print("Adding question mark")
        #Contains question mark 3
        results = collections.Counter(tweet)
        if(results['?'] > 1):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)

        print('Adding question mark')
        #Contains exclamation 4
        if(results['!'] > 1):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)

        print("Adding smiles")
        #Contains emoticon smile 5
        emojis = "😊"
        indiTweetFeature.append(tweet.count(emojis))
        #print(indiTweetFeature)

        print("!?")
        #Contains multiple question or exclamation marks 6
        if results['?'] and results['!'] > 1:
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)

        print("Adding pronouns")
        #Contains pronoun first, sceond, third 7
        pronouns = ["i", "he", "him," "she", "her", "they", "them", "it", "you"]
        tweetSplit = tweet.split(" ")
        for x in range(len(tweetSplit)):
            if(tweetSplit[x].lower() in pronouns):
                indiTweetFeature.append(1)
                break;
            if(x == len(tweetSplit)-1):
                indiTweetFeature.append(0)
        #print(indiTweetFeature)    
        
        print("Adding uppercase")
        #Count uppercase letters 8
        upperLetterCount = 0
        for letter in tweet:
            if(letter.isupper()):
                upperLetterCount += 1
        indiTweetFeature.append(upperLetterCount)
        #print(indiTweetFeature)

        print("Adding url")
        #Number of URLs 9
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet) 
        indiTweetFeature.append(len(url))
        #print(indiTweetFeature)
        
        print("mention")
        #Contains user mention 10
        result = re.findall("(^|[^@\w])@(\w{1,15})", tweet)
        if(len(result) >= threshold):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)

#         print("hashtags")
#         #Contains hashtags 11
#         for x in range(len(tweetSplit)):
#             for letter in range(len(tweetSplit[x])):
#                 if(tweetSplit[x][letter] == "#"):
#                     indiTweetFeature.append(1)
#                     break
#             if(x == len(tweetSplit)-1):
#                 indiTweetFeature.append(0)
#             else:
#                 break
#         print(indiTweetFeature)
     
        #Contains stock symbol
#         file = open('../../Data/nasdaqlisted.txt', 'r')
#         filecontents = file.readlines()
#         stock_symbols = []
#         for line in filecontents:
#             splitline = line.split("|")
#             stock_symbols.append(splitline[0])
            
#         del stock_symbols[0]
        
#         for word in tweetSplit:
#             if(word in stock_symbols):
#                 indiTweetFeature.append(1)
#             else:
#                 indiTweetFeature.append(0)
                
        #Reweet
        
        #Day
        
        print("Adding sentiment")
        #Sentiment 12, 13, 14
        positiveWords = 0
        negativeWords= 0
    
    
        tweetSentiment = TextBlob(tweet)
        sentimentScore = tweetSentiment.sentiment[0]
    
        for word in tweetSentiment:
            word = TextBlob(word)
            if(word.sentiment.polarity) > 0:
                positiveWords = positiveWords + 1
            elif(word.sentiment.polarity) < 0:
                negativeWords = negativeWords + 1
        indiTweetFeature.append(positiveWords)
        indiTweetFeature.append(negativeWords)
        indiTweetFeature.append(sentimentScore)
        #print(indiTweetFeature)
        np.append(tweetsFeatures, np.array(indiTweetFeature), axis = 0)
        #print(len(indiTweetFeature))
        print("Finished feature extraction for tweet {} of {}".format(count, len(tweets)))
        count += 1
    return tweetsFeatures